# 01 Logistic Recession Classifier

Logistic regression, ROC/PR, and threshold tuning.


## Table of Contents
- Train/test split
- Fit logistic
- ROC/PR
- Threshold tuning


## Why This Notebook Matters
Classification notebooks turn the recession label into a **probability model**.
You will learn how to evaluate rare-event prediction and how to choose thresholds intentionally.


## What You Will Produce
- (no file output; learning/analysis notebook)

## Success Criteria
- You can explain what you built and why each step exists.
- You can run your work end-to-end without undefined variables.

## Common Pitfalls
- Running cells top-to-bottom without reading the instructions.
- Leaving `...` placeholders in code cells.
- Reporting only accuracy on imbalanced data.
- Using threshold=0.5 by default without considering costs.

## Matching Guide
- `docs/guides/03_classification/01_logistic_recession_classifier.md`



## How To Use This Notebook
- This notebook is hands-on. Most code cells are incomplete on purpose.
- Complete each TODO, then run the cell.
- Use the matching guide (`docs/guides/03_classification/01_logistic_recession_classifier.md`) for deep explanations and alternative examples.
- Write short interpretation notes as you go (what changed, why it matters).



## Environment Bootstrap
Run this cell first. It makes the repo importable and defines common directories.


In [ ]:
from __future__ import annotations

from pathlib import Path
import sys


def find_repo_root(start: Path) -> Path:
    p = start
    for _ in range(8):
        if (p / 'src').exists() and (p / 'docs').exists():
            return p
        p = p.parent
    raise RuntimeError('Could not find repo root. Start Jupyter from the repo root.')


PROJECT_ROOT = find_repo_root(Path.cwd())
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
SAMPLE_DIR = DATA_DIR / 'sample'

PROJECT_ROOT



## Goal
Train a logistic regression classifier for next-quarter recession and interpret coefficients/odds.


## Your Turn: Fit model


In [ ]:
# TODO: Build train/test split by time
# Fit StandardScaler + LogisticRegression
# Evaluate metrics and plot ROC/PR
...


## Your Turn: Threshold tuning


In [ ]:
# TODO: Evaluate at thresholds 0.3, 0.5, 0.7
# Discuss precision/recall tradeoffs
...


## Checkpoint (Self-Check)
Run a few asserts and write 2-3 sentences summarizing what you verified.



In [ ]:
# TODO: After you build X/y and split by time, validate the split.
# Example (adjust variable names):
# assert X_train.index.max() < X_test.index.min()
# assert y_train.index.equals(X_train.index)
# assert y_test.index.equals(X_test.index)
# assert not X_train.isna().any().any()
# assert not X_test.isna().any().any()
...



## Extensions (Optional)
- Try one additional variant beyond the main path (different features, different split, different model).
- Write down what improved, what got worse, and your hypothesis for why.



## Reflection
- What did you assume implicitly (about timing, availability, stationarity, or costs)?
- If you had to ship this model, what would you monitor?



## Solutions (Reference)

Try the TODOs first. Use these only to unblock yourself or to compare approaches.

<details><summary>Solution: Train/test split</summary>

```python
import pandas as pd
import numpy as np

df = pd.read_csv(SAMPLE_DIR / 'macro_quarterly_sample.csv', index_col=0, parse_dates=True).dropna()
target = 'target_recession_next_q'
drop_cols = {target, 'recession', 'GDPC1', 'gdp_growth_qoq', 'gdp_growth_qoq_annualized', 'gdp_growth_yoy'}
X = df[[c for c in df.columns if c not in drop_cols]].astype(float)
y = df[target].astype(int)
split = int(len(df)*0.8)
X_tr, X_te = X.iloc[:split], X.iloc[split:]
y_tr, y_te = y.iloc[:split], y.iloc[split:]
```

</details>

<details><summary>Solution: Fit logistic</summary>

```python
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from src import evaluation

clf = Pipeline([('scaler', StandardScaler()), ('m', LogisticRegression(max_iter=5000))])
clf.fit(X_tr, y_tr)
p = clf.predict_proba(X_te)[:,1]
evaluation.classification_metrics(y_te.to_numpy(), p)
```

</details>

<details><summary>Solution: ROC/PR</summary>

```python
from sklearn.metrics import roc_curve, precision_recall_curve
from src import plots

fpr, tpr, _ = roc_curve(y_te, p)
prec, rec, _ = precision_recall_curve(y_te, p)
plots.plot_roc_curve(fpr, tpr)
plots.plot_pr_curve(rec, prec)
```

</details>

<details><summary>Solution: Threshold tuning</summary>

```python
from src import evaluation
for thr in [0.3, 0.5, 0.7]:
    print(thr, evaluation.classification_metrics(y_te.to_numpy(), p, threshold=thr))
```

</details>

